<a href="https://colab.research.google.com/github/logicmohe/Document-Automation/blob/master/SPY_Cover_Put.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
class HourData:
  ''' Data in current hour '''
  def __init__(self, timeStamp, open, high, low, close, volume):
    self.timeStamp = timeStamp
    self.open = open
    self.high = high
    self.low = low
    self.close = close
    self.volume = volume
  def test(self):
    print(self.timeStamp)

class DailyData:
  ''' Data in selected date '''
  def __init__ (self, hourlyObj, currentDay):
    self.hourlyObj = hourlyObj
    self.currentDay = currentDay
    self.getMarketHigh()
    self.getMarketLow()
    self.getMarketDiff()
    self.getDayData()
  def test(self):
    print(self.currentDay)

  def getMarketHigh(self):
    self.marketHigh = 0.0
    for obj in self.hourlyObj:
      if float(obj.high)> self.marketHigh:
        self.marketHigh = float(obj.high)
  def getMarketLow(self):
    self.marketLow = 1000.0
    for obj in self.hourlyObj:
      if float(obj.high) < self.marketLow:
        self.marketLow = float(obj.high)
  def getMarketDiff(self):
    self.marketDiff = round(self.marketHigh - self.marketLow, 2)
    self.marketDiffPerc = round(self.marketDiff * 100 / float(self.hourlyObj[11].open),2)
  def getDayData(self):
    self.open = self.hourlyObj[11].open
    self.close = self.hourlyObj[4].close
    self.preClose = self.hourlyObj[4].open

In [ ]:
import requests
r = requests.get('https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=SPY&interval=60min&slice=year1month3&adjusted=false&apikey=X1FABM5B1DPXGUA7')
hourlyObj = [] # List of hour object
dailyObj = [] # List of daily object
hour = 22
currentDay = "1970-01-01"
for hourRecord in r.text.splitlines():
  hour -= 1
  # Remove the first argu line
  if hour == 21:    continue
  # Rotate the day time 5:00 - 20:00
  if hour == 4:
    hour = 20
    dailyObj.append(DailyData(hourlyObj, currentDay))
    hourlyObj = []
      
  hourlyObj.append(HourData(
      hourRecord.split(',')[0], hourRecord.split(',')[1], hourRecord.split(',')[2], \
      hourRecord.split(',')[3], hourRecord.split(',')[4], hourRecord.split(',')[5]  ))
  currentDay = hourRecord.split(",")[0][0:10]

for obj in dailyObj:
  obj.test()
  print("Day Data - open " + obj.open + " close " + obj.close)
  print("Preclose is " + obj.preClose)
  print("Daily Diff is " + str(obj.marketDiff))
  print("Percentage is " + str(obj.marketDiffPerc) + "%")